In [1]:
import pandas as pd
import logging
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from helpers import RateLimiter, detect_endpoint_for_url
TARGET_REGISTRY_PATH='./data/avature_tenants.csv'
OUTPUT_JSON='./data/site_configs_raw.json'
OUTPUT_CSV='./data/site_configs.csv'

In [2]:
# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [3]:
# Configuration
MAX_WORKERS=10
REQUESTS_PER_SECOND = 20  # Global rate limit

In [4]:
progress_lock = Lock()
progress_counter = {'processed': 0, 'success': 0, 'partial': 0, 'failed': 0}

In [5]:
# Initialize rate limiter
rate_limiter = RateLimiter(REQUESTS_PER_SECOND)

In [6]:
def process_row_with_progress(args, rate_limiter):
    """Wrapper for thread execution with progress tracking"""
    idx, row, total = args
    
    result = detect_endpoint_for_url(
        tenant=row['tenant'],
        url=row['full_url'],
        is_career_page=row['is_career_page'],
        rate_limiter=rate_limiter
    )
    
    # Update progress (thread-safe)
    with progress_lock:
        progress_counter['processed'] += 1
        if result['status'] == 'success':
            progress_counter['success'] += 1
            logger.info(f"[{progress_counter['processed']}/{total}] "
                       f"{result['tenant']}: {result['type']}")
        elif result['status'] == 'partial':
            progress_counter['partial'] += 1
            logger.info(f"[{progress_counter['processed']}/{total}] "
                       f"{result['tenant']}: {result['type']}")
        else:
            progress_counter['failed'] += 1
        
        if progress_counter['processed'] % 50 == 0:
            p = progress_counter['processed']
            s = progress_counter['success']
            logger.info(f"Progress: {p}/{total} | Success: {s} ({s/p*100:.1f}%)")
    
    return result

In [7]:
# Load tenant registry
logger.info("Loading tenant registry...")
df_tenants = pd.read_csv(TARGET_REGISTRY_PATH)

# Prioritize career pages first
df_career = df_tenants[df_tenants['is_career_page'] == True].copy()
total=len(df_career)

logger.info(f"Total URLs: {len(df_tenants)}")
logger.info(f"Career pages: {len(df_career)}")

# Start with career pages (highest yield)
logger.info("\n" + "="*60)
logger.info("PHASE 1: Detecting endpoints for career pages")
logger.info("="*60 + "\n")

start_time=time.time()
# Prepare arguments for parallel processing
tasks = [(idx, row, total) for idx, row in df_career.iterrows()]

# Execute in parallel
configs = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(process_row_with_progress, task,rate_limiter) for task in tasks]
    
    for future in as_completed(futures):
        try:
            result = future.result()
            configs.append(result)
        except Exception as e:
            logger.error(f"Thread exception: {e}")

elapsed = time.time() - start_time

#Convert to DataFrame
df_configs = pd.DataFrame(configs)
# Save intermediate results
df_configs.to_json('./data/site_configs_raw.json', orient='records', indent=2)
df_configs.to_csv('./data/site_configs.csv', index=False)

2026-02-08 13:11:44,821 - INFO - Loading tenant registry...
2026-02-08 13:11:44,886 - INFO - Total URLs: 2546
2026-02-08 13:11:44,888 - INFO - Career pages: 855
2026-02-08 13:11:44,890 - INFO - 
2026-02-08 13:11:44,890 - INFO - PHASE 1: Detecting endpoints for career pages
2026-02-08 13:11:44,892 - INFO - ============================================================

2026-02-08 13:11:45,048 - WARNING - Error fetching https://abbvie.avature.net/talent: HTTPSConnectionPool(host='abbvie.avature.net', port=443): Max retries exceeded with url: /talent (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001D82A99F6D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
2026-02-08 13:11:45,072 - WARNING - Error fetching https://abinbev.avature.net/eutalent: HTTPSConnectionPool(host='abinbev.avature.net', port=443): Max retries exceeded with url: /eutalent (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001D8

In [8]:
# Print summary
logger.info("\n" + "="*60)
logger.info("DETECTION SUMMARY")
logger.info("="*60)

total = len(df_configs)
success = len(df_configs[df_configs['status'] == 'success'])
partial = len(df_configs[df_configs['status'] == 'partial'])
failed = len(df_configs[df_configs['status'] == 'failed'])

logger.info(f"\nTotal URLs processed: {total}")
logger.info(f"Success: {success} ({success/total*100:.1f}%)")
logger.info(f"Partial: {partial} ({partial/total*100:.1f}%)")
logger.info(f"Failed: {failed} ({failed/total*100:.1f}%)")

2026-02-08 13:15:55,004 - INFO - 
2026-02-08 13:15:55,004 - INFO - DETECTION SUMMARY
2026-02-08 13:15:55,010 - INFO - ============================================================
2026-02-08 13:15:55,019 - INFO - 
Total URLs processed: 855
2026-02-08 13:15:55,020 - INFO - Success: 179 (20.9%)
2026-02-08 13:15:55,020 - INFO - Partial: 9 (1.1%)
2026-02-08 13:15:55,025 - INFO - Failed: 667 (78.0%)


In [9]:
if success > 0:
    logger.info("\nEndpoint Types Detected:")
    type_counts = df_configs[df_configs['status'] == 'success']['type'].value_counts()
    for endpoint_type, count in type_counts.items():
        logger.info(f"{endpoint_type}: {count}")

2026-02-08 13:15:55,046 - INFO - 
Endpoint Types Detected:
2026-02-08 13:15:55,053 - INFO - search_jobs_html: 179
